# Image Instance Segmentation with Mask R-CNN

---

# Abstract

Deep Learning 기술을 Computer Vision분야에 적용함으로써 많은 전문적인 분야들이 파생되었다. classification, segmentation, object detection, GAN 등이 이에 해당하는데, 특히 segmentation과 object detection을 접목해서 성능을 높인 ‘Mask R-CNN’ 모델을 깊이 연구하기로 한다. 구체적인 사항으로는 Mask R-CNN의 논문을 리뷰하고 토의하는 방식을 통해 모델을 이해한다. 그리고 Keras를 통해 모델을 구현하여 COCO dataset에 적용시킴으로써 모델의 성능을 평가하고 논문의 내용과 비교분석한다.

# Object Detection

<img src="https://i.imgur.com/dXc6g6c.png" width="100%">

Mask R-CNN을 이해하기 위해서는 그 기저에 깔린 Object Detection과 Segmentation이 무엇인지 먼저 알 필요가 있다. 위 그림을 살펴보자.<br>
먼저 Semantic Segmentation의 경우, 각각의 픽셀에 대하여 해당 픽셀이 어떤 객체에 속하는지를 분류한다. 위 그림의 경우 각 픽셀이 GRASS, CAT, TREE, SKY 중에 어떤 것에 해당하는지를 분류하고 값을 다르게 매긴 것을 확인할 수 있다.<br>
Classification 그리고 Localization의 경우, Classification은 단순히 사진 전체가 어떤 객체를 의미하는지를 분류하고, Localization은 그 객체의 Bounding Box를 추출해낸다. 이 때 Classification은 분류 문제에 해당하고, Localization은 회귀 문제에 해당한다. 단일 객체만을 분류 및 Bounding Box를 할 수 있다는 것이 특징이다.<br>
Object Detection의 경우, 여러 객체에 대하여 Classification과 Bounding Box를 추출해낸다. 특히 위의 그림처럼 같은 객체(DOG)라도 각자 Classification 및 Bounding Box를 추출해낸다는 것이 특징이다. 따라서 앞의 문제에 비해 고난이도의 문제라고 할 수 있다.<br>
마지막으로 Instance Segmentation의 경우, 모든 객체(마찬가지로 같은 객체라도 각자 수행)에 대하여 Classification 및 Bounding Box를 진행하고, 여기서 더 나아가 각 Bounding Box내에 Segmentation을 수행한다. 따라서 위의 그림처럼 객체를 넘어서 모든 인스턴스를 Classification하고 Segmentation한 결과를 확인할 수 있다. 우리가 살펴볼 Mask R-CNN은 이에 해당한다.

Mask R-CNN을 잘 이해하기 위해서는 결국 그 기저에 깔린 수많은 내용들을 이해해야 할 것이다. 이 모델은 갑자기 튀어나온 것이 아니라 사실 Object Detection을 위해 만들어진 모델에서 출발했다. 따라서 R-CNN 시리즈 모델이라고도 할 수 있는, R-CNN, Fast R-CNN, Faster R-CNN, Mask R-CNN의 주요 특징들과 변천과정을 이해한다면 Mask R-CNN을 더욱 잘 이해할 수 있을 것이다.

# R-CNN(2013)

### Model

<img src="https://i.imgur.com/HIpL2ks.png" width="100%">

<img src="https://i.imgur.com/jrU53ke.png" width="100%">

R-CNN의 작동원리와 구조는 위 두 그림과 같다.<br>
먼저 Input Image에서 약 2000여장의 이미지를 'Selective Search' 알고리즘을 통해 추출한다. 이 알고리즘에 대한 자세한 내용은 아래에서 설명한다. 이 이미지는 우리가 관심있어하는 정보이기에 'RoI(Region of Interest)'라고 부른다. 추출한 RoI를 각각의 CNN 모델에 입력하기 위해 Input Size를 동일하게 맞춰준다. 이 과정을 'Warp'이라고 한다. 그리고 각 CNN 모델은 2개의 연산으로 나뉘게 되는데, 하나는 분류를 위한 Classification이고, 나머지 하나는 Bounding Box를 정교하게 추출하기 위한 Regression이다. Bounding Box Regression은 Neural Network로 구성되어 있는 반면, Classification은 SVM이라는 다른 모델을 사용한다. 따라서 CNN 모델의 마지막 feature map을 SVM에 적용하여 Classification을 수행한다.

### Selective Search

<img src="https://i.imgur.com/V2dnlCS.png" width="100%">

Selective Search란 Object Detection에서 입력된 이미지의 후보 영역을 추천할 때 사용되었던 방법으로 R-CNN 등의 논문들이 탁월한 성능을 보여주면서 주목받게 되었다. 특히 이는 2가지 방법을 조합하여 수행하는데, Exhaustive Search와 Segmentation이 이에 해당한다. 전자의 경우 모든 가능한 객체의 위치를 찾아내는 것을 의미하고, 후자의 경우 이미지의 구조적 특징(색상, 무늬, 크기, 모양)을 사용하여 후보 영역을 추출하는 것을 의미한다.

<img src="https://i.imgur.com/71hypJX.png" width="100%">

알고리즘은 다음과 같은 방식으로 수행된다.<br><br>
1. Efficient Graph-Based Image Segmentation(Felzenszwalb)을 사용하여 초기 후보 영역을 다양한 크기와 비율로 생성한다.
2. 그리디 알고리즘을 통해 비슷한 영역을 반복적으로 통합한다.
    * 처음에 모든 영역에 대해 유사도를 계산하여 similarity set S를 생성한다.
    * S에서 가장 큰 유사도 값을 가진 r(i), r(j)에 대해 통합한다.
    * r(i), r(j)의 유사도 값은 S로부터 제거한다.
    * 통합된 새로운 영역(r(t))과 인접한 영역들에 대해 유사도(S(t))를 계산한다.
    * S와 R에 유사도(S(t))와 통합된 새로운 영역(r(t))을 추가한다.
3. 최종적으로 하나의 영역이 만들어질 때까지 2번을 반복한다.

### R-CNN Training

<img src="https://i.imgur.com/6yOt47e.png" width="100%">

* Pre-train a ConvNet(AlexNet) for ImageNet classification dataset
* Fine-tune for object detection(softmax + log loss)
* Cache feature vectors to disk
* Train post hoc linear SVMs(hinge loss)
* Train post hoc linear bounding-box regressors(squared loss)

모델 구성을 조금 더 설명하자면 ImageNet classification dataset으로 Pre-train된 모델을(위의 그림에서는 AlexNet을 예로 들고 있다)을 ConvNet으로 활용한다. 이 때 AlexNet의 마지막 layer를 Obejct Detection을 위한 class의 수로 수정하여 fine tuning을 수행한다. 이후 마지막 layer의 이전 layer를 디스크에 따로 저장을 해놓는다. 그리고 그 저장된 데이터를 SVM에 학습시킨다.<br><br>
* fine tuning: 기존에 학습되어져있는 모델을 기반으로 아키텍처를 새로운 목적(나의 이미지 데이터에 맞게)에 맞게 변형하고 이미 학습된 모델 weights로부터 학습을 업데이트하는 방법을 말한다. 이는 모델의 파라미터를 미세하게 조정하는 행위로써 특히 딥러닝에서는 이미 존재하는 모델에 추가 데이터를 투입하여 파라미터를 업데이트하는 것을 말한다.

Question) bounding-box regressor는 어떤 layer를 통해 학습하는가? 그리고 이 과정은 end-to-end인가?(PPT 12쪽에 따르면 update되지 않는다고 쓰여져 있음)

### Bounding-Box Regression

<img src="https://i.imgur.com/5ikzHbY.png" width="100%">

<img src="https://i.imgur.com/dKxKXMX.png" width="100%">

<img src="https://i.imgur.com/zjXZJ8Q.png" width="100%">

Bounding-Box Regression의 작동원리는 Bounding Box의 중심 x좌표 x, y좌표 y, 높이 h, 너비 w를 예측한 P와 실제값 G를 줄이는 방식으로 진행된다. 특히 P와 G의 차이를 나타내는 t와 우리가 조절해야 할 대상인 d(P)의 간격을 최소화하는 방향으로 이루어진다.

Question) Bounding-Box Regression 위의 내용이 맞는지 여부?

### Problems of R-CNN

R-CNN의 문제점은 다음과 같다.<br><br>
1. Slow at test-time<br>
R-CNN의 구조를 보면 처음 Selective Search를 통해 얻은 2000여장의 Region Proposal을 각 ConvNet에 모두 입력시켜야한다. 즉 2000여개의 ConvNet에 대입시켜야한다는 뜻이다. 이는 test시에 굉장히 많은 시간을 필요로 하게 된다. 특히 논문에 게재된 내용에 따르면 test시 image 한 장당 GPU(K40)를 이용하여 13초의 시간을, CPU를 이용하여 53초의 시간을 필요로 했다.
2. SVM and regressors are post-hoc<br>
ConvNet과 SVM, Regressor는 별개의 모델로써 end-to-end 학습을 수행할 수 없다. 따라서 SVM과 Regressor에서 학습한 내용을 ConvNet에 전달할 수 없다.
3. Complex multistage training pipeline(84 hours using K40 GPU)<br>
R-CNN의 구조는 multistage training pipeline으로써 많은 모델을 한꺼번에 가지고 있다. ConvNet, SVMs, Bounding-Box Regressor가 이에 해당하는데, 이들을 모두 학습시켜야하기때문에 굉장히 복잡한 구조를 가지고 있다고 할 수 있다.

# Fast R-CNN(2015)

위의 R-CNN의 문제점들을 보완해서 나온 모델이 Fast R-CNN이다. 아래 그림을 통해 자세히 살펴보자.

### Model

<img src="https://i.imgur.com/gM5oGFt.png" width="100%">

<img src="https://i.imgur.com/m31U15k.png" width="100%">

Selective Search를 통해 RoI를 추출하는 것은 동일하다. 하지만 R-CNN과는 달리 이미지 전체를 ConvNet에 바로 입력시킨다. 이 때 Input Image가 ConvNet을 거치면서 사이즈가 pooling되는데 RoI 또한 같이 pooling된다고 볼 수 있다. 이를 RoI projection이라고 한다. 이후 각각의 Bounding Box를 poolin하여 같은 사이즈로 맞춰주는 RoI pooling을 수행한다. 이는 Fully Connected Layer에 동일한 사이즈로 입력시켜주기 위함이다. RoI pooling으로 인해 모두 동일한 사이즈를 갖게 된 데이터는 FC를 거쳐 RoI feature vector가 되고, 또 다시 병렬적인 FC를 거쳐 Softmax(classification)와 Bounding Box Regression을 수행하는 구조로 이루어져있다. 즉 Fast R-CNN의 핵심은 RoI pooling이라고 할 수 있겠다.

### RoI Pooling

<img src="https://i.imgur.com/BV8xO56.png" width="100%">

<img src="https://i.imgur.com/zQww6ym.png" width="100%">

ConvNet을 통과한 feature map에 존재하는 Bounding Box를 다음 FC에 입력하기 위해 모두 같은 사이즈로 만들어주는 것을 RoI Pooling이라고 했다. 구체적으로 어떻게 작동하는지 살펴보자.<br>
예를 들어 FC에 입력할 사이즈를 7x7로 정해놓았다고 가졍하자. 그럼 ConvNet을 통과한 feature map에 존재하는 모든 RoI의 사이즈를 7x7로 만들어주어야한다. 어떤 RoI의 사이즈가 21x14라고 하면, (3,2)의 stride를 가진 3x2 max pooling을 통해 7x7로 만들어줄 수 있다. 마찬가지로 또 다른 RoI의 사이즈가 35x42라고 하면, (5,6)의 stride를 가진 5x6 max pooling을 통해 7x7로 만들어줄 수 있다. 이처럼 모든 RoI에 각각에 맞는 stride를 적용하여 max pooling을 한다면 모두 같은 사이즈로 만들어 줄 수 있다. 이를 보기 편하게 설명하기 위해 아래와 같이 정리할 수 도 있다.<br><br>
RoI in Conv feature map : 21x14 -> 3x2 max pooling with stride(3, 2) -> output : 7x7<br>
RoI in Conv feature map : 35x42 -> 5x6 max pooling with stride(5, 6) -> output : 7x7

### Training & Testing

<img src="https://i.imgur.com/vNfzMVF.png" width="100%">

<img src="https://i.imgur.com/DatXybA.png" width="100%">

Fast R-CNN을 훈련시킴에 있어서 Loss Function을 눈여겨 볼 만하다. L(cls)는 우리가 흔히 알고 있는 Cross Entropy를 사용했고, L(loc)은 새롭게 함수를 정의해놓았다. 특히 u가 1 이상일 경우를 설정해놓았는데, 이는 u가 0인 경우, 즉 배경인 경우에는 L(loc)을 고려하지 않겠다는 것을 의미한다. 이는 엉뚱한 곳에 RoI를 잡은 경우 Bounding Box Regression에 대한 loss는 구하지 않도록 해준다. 특히 IoU에 대한 임계값을 설정하면 그 임계값 이상인 RoI에 대해서만 loss를 구하게 된다.

### R-CNN vs. SPP-net vs. Fast R-CNN

<img src="https://i.imgur.com/9KEPmyF.png" width="100%">

그렇다면 Fast R-CNN은 얼마나 성능이 향상되었을까? 위의 그래프를 통해 확인할 수 있다. 기존 R-CNN에 비해 Training time과 Test time을 굉장히 단축시킨 것을 확인할 수 있다. 특히 눈여겨 봐야 할 점은 오른쪽 그래프의 Fast R-CNN이다. 파란색 그래프는 Region Proposal를 포함시킨 시간이고, 빨간색 그래프는 포함시키지 않은 시간이다. 즉, 이 두 시간의 차이는 Region Proposal을 구하는 시간인 Selective Search를 의미한다. 이 시간이 전체 시간의 대부분을 차지하고 있음을 확인할 수 있다. 이는 앞에서도 언급했다시피 CPU상에서 작동하기 때문이다.

### Problems of Fast R-CNN

Fast R-CNN의 문제점은 다음과 같다.<br><br>
1. Out-of-network region proposals are the test-time computational bottleneck<br>
Region Proposal, 즉 Selective Search가 Network의 바깥에 존재하여 시간이 굉장히 오래 걸린다.
2. Is it fast enough?<br>
과연 Fast R-CNN의 성능을 빠르다고 할 수 있는가는 생각해봐야한다. Real-Time을 적용하기 위해서는 2.3초는 빠르다고 할 수는 없을것이다.

# Faster R-CNN(2015)

### Model

R-CNN, Fast R-CNN은 CPU에서 수행하는 Selective Search때문에 많은 시간을 필요로 했다. 이를 마지막 convolution layer 이후에 RPN(Region Proposal Network)을 사용하여 GPU를 사용해보자는 것이 Faster R-CNN의 아이디어이다. 즉, Faster R-CNN을 한마디로 표현하면 RPN + Fast R-CNN이라고 할 수 있다.

<img src="https://i.imgur.com/ytqmSLH.png" width="100%">

특히 위 그림과 같이 RPN에 사용될 ConvNet과 Fast R-CNN에 사용될 ConvNet을 같은 것으로 사용하여 정보를 공유해보자는 목표를 가지고 있다.

### RPN

Question) 설명 필요

### Positive/Negative Samples

Question) 설명 필요

### RPN Loss Function

Question) 설명 필요

### 4-Step Alternating Training

Question) 설명 필요

### Results

<img src="https://i.imgur.com/In4IVHA.png" width="100%">

위의 표는 R-CNN, Fast R-CNN, Faster R-CNN의 정확도와 성능을 비교하고있다. 이미지 1장당 test time을 0.2초까지 줄였으며, 이는 R-CNN에 비해 250배 향상된 속도이다.

<img src="https://i.imgur.com/1Jmc0o3.png" width="100%">

또한 위의 표에서 VGG를 사용한 모델을 살펴보자. SS + Fast R-CNN은 Selective Search를 사용한 모델로써, 기존 Fast R-CNN을 의미한다. 그리고 그 아래 RPN + Fast R-CNN은 Faster R-CNN을 의미한다. 즉, CPU를 사용했던 Selective Search를 GPU를 사용하는 RPN으로 바꾸면 proposal부분에서 시간이 굉장히 단축됨을 확인할 수 있다.

### Problems of Faster R-CNN

Faster R-CNN의 문제점은 다음과 같다.<br>
1. RoI Pooling has some quantization operations

Question) RPN 이해 후에 다시 문제점 살펴볼 것

# Mask R-CNN

# Reference

[Deep Learning] pre-training 과 fine-tuning (파인튜닝)
* https://eehoeskrap.tistory.com/186

Object recognition을 위한 선택적 검색
* https://murra.tistory.com/25